In [9]:
from jupyterscad import view
from  solid2 import * 
import os 
import math
import gradio as gr

from const import *

In [16]:
debug = False
should_render = True
filename = "body"

# define base dimensions
# ring size
base_r = 15.3
# height of the ring
base_d = 4
# width of ring material
base_w = 1
# gap between inner and outer circle
gap = 1
# manufacturing tolerance
tolerance = 0
switch_mat_w = 0.1
cut = 0.5
lip = 0.2
lip_a = 0.2
lip_b = 0.2
mag_count = 16
base_inner_r = base_r-base_w
mag_a = 1
mag_b = base_d
mag_cut_a = mag_a + cut
mag_cut_b = mag_b + 2
chip_esp_a =  4
chip_esp_b =  1
chip_esp_cut_a = chip_esp_a + 2*cut

# minimal width of material for cutouts etc.
material_min = 0.2
switch_base_offset = 5

inner_r,inner_d,inner_w = [base_r,base_d,base_w]
outer_r,outer_d,outer_w = [inner_r+inner_w+gap,inner_d,inner_w]
switch_base_r,switch_base_d,switch_base_w = [base_r+switch_base_offset,base_d,switch_mat_w]
#if outer_<= mag_a:
outer_r += material_min
outer_w += material_min

obj = []
set_global_fn(30)

In [17]:
inner_base = cylinder(
    inner_d,
    inner_r+inner_w,
    inner_r+inner_w
)
inner_cut = cylinder(
    inner_d+2,
    inner_r,
    inner_r
)
outer_base = cylinder(
    outer_d,
    outer_r+outer_w,
    outer_r+outer_w
)
outer_cut = cylinder(
    outer_d+2,
    outer_r,
    outer_r
)
outer_top = cylinder(
    lip_b,
    outer_r+outer_w,
    outer_r+outer_w
)
outer_top_cut = cylinder(
    lip_b+2,
    inner_r+gap,
    inner_r+gap
)

switch_base = cylinder(
    switch_base_d,
    switch_base_r+switch_base_w,
    switch_base_r+switch_base_w
)
switch_base_cut = cylinder(
    switch_base_d+2,
    switch_base_r,
    switch_base_r
)

switch_base_union = cube(
    switch_base_r,
    switch_base_r,
    switch_base_d
)


In [18]:
chip_esp = cube(chip_esp_a,chip_esp_a,chip_esp_b).rotate(90,0,0).translate(
    -chip_esp_a/2,inner_r+inner_w-cut,0
)
chip_esp_cut= cube(chip_esp_cut_a,chip_esp_cut_a,chip_esp_b).rotate(90,0,0).translate(
    -chip_esp_a/2 - cut,inner_r+inner_w-cut,0
)
magnets = []
magnet_cuts = []

mag_base = cube(mag_a,mag_a,mag_b).translate(
    outer_r,-mag_a/2,0
).translate(
    -cut/2,0,0
)
mag_cut = cube(mag_cut_a,mag_cut_a,mag_cut_b).translate(
    outer_r,-mag_cut_a/2,-1
).translate(
    outer_w-cut-gap-material_min,0,0
)

for magnet in range(0, mag_count):
    magnets += mag_base.rotate(0,0,magnet*360/mag_count+360/mag_count/2)
    magnet_cuts += mag_cut.rotate(0,0,magnet*360/mag_count+360/mag_count/2)



outer_top_all = (outer_top - outer_top_cut.translate(0,0,-1)).translate(0,0,base_d)
inner = inner_base - inner_cut.translate(0,0,-1) - chip_esp_cut
outer = outer_base - outer_cut.translate(0,0,-1) - magnet_cuts 
switch = switch_base - switch_base_cut.translate(0,0,-1) 
#outer = outer_base - outer_cut.translate(0,0,-1)

switch_base = switch.translate(-(switch_base_r-switch_base_d-inner_r+base_w+gap),-(switch_base_r-switch_base_d-inner_r+base_w+gap), 0)
switch_n = (switch_base * switch_base_union)


switches = []
bar_w = 0.3
#bar_angle = math.pow(math.atan2(base_d, outer_r)*math.pi,2)
bar_a= math.pow(base_r,2)
bar_b= base_d
bar_angle1 = math.atan2(bar_b,bar_a)
#bar_angle = math.degrees(bar_angle1)
bar_angle = 6.6
bar_offset = (base_d-bar_w)/2
bar_n  = cube(110,110, bar_w).rotate(0,bar_angle,0).rotate(bar_angle,0,0).translate(0,0,bar_offset)
bar_m  = cube(110,110, bar_w).rotate(0,-bar_angle,0).rotate(-bar_angle,0,0).translate(0,0,bar_offset)
bar_t  = cube(110,110, bar_w).translate(0,0,base_d-bar_w)
bar_b  = cube(110,110, bar_w).translate(0,0,0)
bar_center = cube(bar_w,bar_w*2,base_d).translate(outer_r,0,0)
bars = []

bar_all = (bar_n * switch_n) + (bar_m * switch_n) + (bar_t * switch_n) + (bar_b * switch_n)
bar_all = bar_all.translate(2*bar_w,2*bar_w,0) + bar_center.translate(0,0,0) + bar_center.translate(0,-2*bar_w,0)

chips = []
chips += chip_esp

for i in range(0,4):
    switches += switch_n.rotate(0,0,i*90)
    bars += bar_all.rotate(0,0,i*90)
    
magnets = magnets.color([0.9,0,0])
inner = inner.color([1,0.1,0.5])
outer = outer.color([0,0.9,0]) 
bars = bars.color([0,0,1]) 
rendered = [ inner,  bars,  chips]
#obj += inner +outer - magnet_cuts + magnets

for i, r in enumerate(rendered):
    if debug:
        obj += r.translate(0,0,-base_d*i)
    else:
        obj += r


def render(obj):
    obj.save_as_scad(f"{filename}.scad")
    obj.save_as_stl(f"{filename}.stl")

if should_render:
    render(obj)

view(obj)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.7, position=(3.0, 5.0,…

In [13]:
def load_mesh(value,mesh_file_name):
    get_base_dims()
    render(obj)
    return mesh_file_name

demo = gr.Interface(
    fn=load_mesh,
    inputs=[
        gr.Slider(0, 20, value=gap, label="gap", info="Choose between 2 and 20"),gr.Model3D(f"/home/m/proj/mousring/{filename}.stl")],
    outputs=gr.Model3D(clear_color=(0.0, 0.0, 0.0, 0.0),  label="3D Model", display_mode="wireframe"),

    examples=[
        [4,f"/home/m/proj/mousring/{filename}.stl"],
    ],
)

In [14]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
